In [33]:
!pip install tensorflow-model-optimization

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [34]:
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import sys

In [35]:
base_path = "/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition"
base_path

'/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition'

In [36]:
uci_har_dataset_path = f'{base_path}/data/UCI_HAR'
uci_har_dataset_path

'/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/data/UCI_HAR'

In [37]:
if base_path not in sys.path:
    sys.path.append(base_path) 

In [38]:
import src.load_uci_har_dataset as uci_har

In [39]:
Xtrain, ytrain, xtest, ytest, labels = uci_har.load_uci_har_dir(uci_har_dataset_path)

In [40]:
import src.get_models as models

In [41]:
import importlib
importlib.reload(models)

<module 'src.get_models' from '/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/src/get_models.py'>

In [42]:
base_model = keras.models.load_model(f'{base_path}/models/cnn_lstm_model.keras')

2025-12-15 17:55:09.678049: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 17:55:09.679007: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 17:55:09.679568: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [43]:
baseline_model_loss, baseline_model_accuracy = base_model.evaluate(
    xtest, ytest)

print('Baseline test accuracy:', baseline_model_accuracy)


2025-12-15 17:55:23.794240: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 17:55:23.795081: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 17:55:23.795615: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

93/93 [==============================] - 3s 29ms/step - loss: 0.2449 - accuracy: 0.9321
Baseline test accuracy: 0.9321343898773193


In [44]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

batch_size = 64
epochs = 100
validation_split = 0.1

num_images = Xtrain.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(base_model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 128, 6)]          0         
                                                                 
 prune_low_magnitude_conv1d_  (None, 128, 64)          3906      
 4 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_batch_n  (None, 128, 64)          257       
 ormalization_6 (PruneLowMag                                     
 nitude)                                                         
                                                                 
 prune_low_magnitude_re_lu_4  (None, 128, 64)          1         
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_conv1d_  (None, 128, 128)         4928

2025-12-15 17:55:37.780921: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 17:55:37.781785: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 17:55:37.782314: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [45]:
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=f'{base_path}/pruned_models_lstm'),
]

model_for_pruning.fit(Xtrain, ytrain,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/100


2025-12-15 17:56:01.213092: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 17:56:01.214071: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 17:56:01.214665: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

104/104 [==============================] - ETA: 0s - loss: 0.3494 - accuracy: 0.8643

2025-12-15 17:56:14.089660: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 17:56:14.090509: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 17:56:14.091037: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

104/104 [==============================] - 14s 102ms/step - loss: 0.3494 - accuracy: 0.8643 - val_loss: 0.6584 - val_accuracy: 0.8043
Epoch 2/100
104/104 [==============================] - 10s 98ms/step - loss: 0.3137 - accuracy: 0.8755 - val_loss: 0.4665 - val_accuracy: 0.8043
Epoch 3/100
104/104 [==============================] - 10s 97ms/step - loss: 0.2774 - accuracy: 0.8960 - val_loss: 1.0630 - val_accuracy: 0.7459
Epoch 4/100
104/104 [==============================] - 10s 97ms/step - loss: 0.3046 - accuracy: 0.8818 - val_loss: 1.3801 - val_accuracy: 0.7215
Epoch 5/100
104/104 [==============================] - 10s 97ms/step - loss: 0.2527 - accuracy: 0.9102 - val_loss: 2.8538 - val_accuracy: 0.7310
Epoch 6/100
104/104 [==============================] - 10s 96ms/step - loss: 0.2503 - accuracy: 0.9076 - val_loss: 2.1044 - val_accuracy: 0.7622
Epoch 7/100
104/104 [==============================] - 10s 99ms/step - loss: 0.2492 - accuracy: 0.9099 - val_loss: 3.8049 - val_accuracy: 0.6

104/104 [==============================] - 10s 98ms/step - loss: 0.1702 - accuracy: 0.9432 - val_loss: 1.4765 - val_accuracy: 0.7717
Epoch 58/100
104/104 [==============================] - 10s 97ms/step - loss: 0.1868 - accuracy: 0.9397 - val_loss: 8.0264 - val_accuracy: 0.6658
Epoch 59/100
104/104 [==============================] - 10s 96ms/step - loss: 0.1755 - accuracy: 0.9418 - val_loss: 5.8709 - val_accuracy: 0.6753
Epoch 60/100
104/104 [==============================] - 10s 96ms/step - loss: 0.1688 - accuracy: 0.9415 - val_loss: 6.2874 - val_accuracy: 0.6902
Epoch 61/100
104/104 [==============================] - 10s 98ms/step - loss: 0.1704 - accuracy: 0.9459 - val_loss: 6.0886 - val_accuracy: 0.7174
Epoch 62/100
104/104 [==============================] - 10s 96ms/step - loss: 0.1673 - accuracy: 0.9479 - val_loss: 5.6067 - val_accuracy: 0.6766
Epoch 63/100
104/104 [==============================] - 10s 97ms/step - loss: 0.1756 - accuracy: 0.9424 - val_loss: 5.0385 - val_accuracy

In [46]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   xtest, ytest, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9321343898773193
Pruned test accuracy: 0.8150661587715149


In [47]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
pruned_keras_file = f'{base_path}/pruned_models_lstm_cnn'
keras.models.save_model(model_for_export, pruned_keras_file , include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

2025-12-15 18:13:31.272843: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:13:31.273477: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:13:31.274161: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:13:32.732354: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:13:32.733441: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:13:32.734109: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:13:33.559235: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:13:33.559824: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:13:33.560486: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:13:34.066724: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:13:34.067447: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:13:34.067951: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

INFO:tensorflow:Assets written to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/pruned_models_lstm_cnn/assets


INFO:tensorflow:Assets written to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/pruned_models_lstm_cnn/assets


Saved pruned Keras model to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/pruned_models_lstm_cnn


In [48]:
import tensorflow as tf
run_model = tf.function(lambda x: model_for_pruning(x))
BATCH_SIZE = 1
STEPS = 128
INPUT_SIZE = 6
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], model_for_pruning.inputs[0].dtype))

# model directory.
MODEL_DIR = f"{base_path}/LSTM_CNN_Pruning"
model_for_pruning.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

2025-12-15 18:13:34.931770: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:13:34.932351: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:13:34.933010: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:13:35.622072: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:13:35.622709: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:13:35.623339: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:13:37.147923: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:13:37.148770: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:13:37.149302: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:13:38.927097: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:13:38.927849: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:13:38.928363: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

INFO:tensorflow:Assets written to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/LSTM_CNN_Pruning/assets


INFO:tensorflow:Assets written to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/LSTM_CNN_Pruning/assets


In [49]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
pruned_tflite_model = converter.convert()

pruned_tflite_file = f"{base_path}/models/cnn_lstm_pruned.tflite"

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

2025-12-15 18:13:45.334942: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 18:13:45.335009: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 18:13:45.335044: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:13:45.774199: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 18:13:45.774266: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 18:13:45.774300: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:13:46.347400: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 18:13:46.347455: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 18:13:46.347488: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Saved pruned TFLite model to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/models/cnn_lstm_pruned.tflite


2025-12-15 18:13:47.641605: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-12-15 18:13:47.641634: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-12-15 18:13:47.642754: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/LSTM_CNN_Pruning
2025-12-15 18:13:47.657545: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2025-12-15 18:13:47.657558: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/LSTM_CNN_Pruning
2025-12-15 18:13:47.706945: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-12-15 18:13:47.841991: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundl

In [50]:
import tempfile
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [51]:
keras_file = f"{base_path}/models/cnn_lstm_baseline.h5"
keras.models.save_model(base_model, keras_file, include_optimizer=False)

In [52]:
print("Size of gzipped TFlite model: %.2f bytes" % (get_gzipped_model_size(keras_file)))

Size of gzipped TFlite model: 273170.00 bytes


In [53]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

quantized_and_pruned_tflite_file = f"{base_path}/models/cnn_lstm_pruned_dynamic.tflite"

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

2025-12-15 18:13:52.656753: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 18:13:52.656822: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 18:13:52.656855: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:13:53.095032: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 18:13:53.095101: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 18:13:53.095137: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:13:53.674077: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 18:13:53.674131: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 18:13:53.674164: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Saved quantized and pruned TFLite model to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/models/cnn_lstm_pruned_dynamic.tflite
Size of gzipped pruned and quantized TFlite model: 312356.00 bytes
Size of gzipped pruned TFlite model: 312335.00 bytes


2025-12-15 18:13:54.396042: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-12-15 18:13:54.396066: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-12-15 18:13:54.396722: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/LSTM_CNN_Pruning
2025-12-15 18:13:54.416481: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2025-12-15 18:13:54.416499: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/LSTM_CNN_Pruning
2025-12-15 18:13:54.484373: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-12-15 18:13:54.628608: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundl

In [54]:
def evaluate_model(interpreter):
    in_det  = interpreter.get_input_details()[0]
    out_det = interpreter.get_output_details()[0]
    in_idx  = in_det["index"]
    out_idx = out_det["index"]
    in_dtype = in_det["dtype"]     
    in_scale, in_zero = in_det["quantization"]
    print("dtype", in_dtype)

    prediction_digits = []
    for i, test_image in enumerate(xtest):
        if i % 1000 == 0:
            print(f"Evaluated on {i} results so far.")

        xi = np.expand_dims(test_image, axis=0).astype(np.float32)

        if in_dtype == np.int8:
            xi_q = np.clip(np.round(xi / in_scale + in_zero), -128, 127).astype(np.int8)
            interpreter.set_tensor(in_idx, xi_q)
        elif in_dtype == np.uint8:
            xi_q = np.round(xi / in_scale + in_zero).astype(np.uint8)
            interpreter.set_tensor(in_idx, xi_q)
        else:
            interpreter.set_tensor(in_idx, xi)

        interpreter.invoke()

        yo = interpreter.get_tensor(out_det["index"])[0]
        prediction_digits.append(int(np.argmax(yo)))

    print('\n')
    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == ytest).mean()
    return accuracy

In [55]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

dtype <class 'numpy.float32'>
Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.


Pruned and quantized TFLite test_accuracy: 0.8286392941974889
Pruned TF test accuracy: 0.8150661587715149


In [56]:
print("CNN-LSTM Pruned model size in MB:", os.path.getsize(f"{base_path}/models/cnn_lstm_pruned.tflite") / float(2**20))

CNN-LSTM Pruned model size in MB: 1.6739387512207031


In [57]:
print("CNN-LSTM Pruned and quantized model size in MB:", os.path.getsize(f"{base_path}/models/cnn_lstm_pruned_dynamic.tflite") / float(2**20))

CNN-LSTM Pruned and quantized model size in MB: 1.6739425659179688
